# Developing Custom Models

Panel ships with a number of custom Bokeh models, which have both Python and Javascript components. When developing Panel these custom models have to be compiled. This happens automatically with `pip install -e .` or `python setup.py develop`, however when runnning actively developing you can rebuild the extension with `panel build panel`. The build command is just an alias for `bokeh build`; see the [Bokeh developer guide](https://docs.bokeh.org/en/latest/docs/dev_guide/setup.html) for more information about developing bokeh models or the [Awesome Panel - Bokeh Extensions Guide](https://awesome-panel.readthedocs.io/en/latest/guides/awesome-panel-extensions-guide/bokeh-extensions.html)

Just like any other Javascript (or Typescript) library Panel defines a `package.json` and `package-lock.json` files. When adding, updating or removing a dependency in the `package.json` file ensure you commit the changes to the `package-lock.json` after running npm install.

## Adding a New Model

This example will guide you through adding a new model. 

We will use the the `ChartJS` model as an example. But you should replace `ChartJS` and similar with the name of your model. 

Here we will add a simple Button model to start with. My experience is that you should start small with a working example and the continue in small, incremental steps. For me it did not work trying to copy a large, complex example and refactoring it when I started out learning about Custom Models.

1. Create a new branch `chartjs`.
2. Add the files and code for a *minimum working model*. 

- Add a Panel `panel/pane/chartjs.py` file
    
```python
import param

from panel.widgets.base import Widget

from ..models import ChartJS as _BkChartJS


class ChartJS(Widget):
    # Set the Bokeh model to use
    _widget_type = _BkChartJS

    # Rename Panel Parameters -> Bokeh Model properties
    # Parameters like title that does not exist on the Bokeh model should be renamed to None
    _rename = {
        "title": None,
    }

    # Parameters to be mapped to Bokeh model properties
    object = param.String(default="Click Me!")
    clicks = param.Integer(default=0)
```

- Add the Panel model to `panel/pane/__init__.py`

```python
from .chartjs import ChartJS
```

- Add a Bokeh `panel/models/chartjs.py` model file

```python
from bokeh.core.properties import Int, String
from bokeh.models import HTMLBox

class ChartJS(HTMLBox):
    """Custom ChartJS Model"""

    object = String()
    clicks = Int()
```

- Add the Bokeh model to `panel/models/__init__.py`

```python
from .chartjs import ChartJS
```

- Add the Bokeh model to ``

```typescript
// See https://docs.bokeh.org/en/latest/docs/reference/models/layouts.html
import { HTMLBox, HTMLBoxView } from "@bokehjs/models/layouts/html_box"

// See https://docs.bokeh.org/en/latest/docs/reference/core/properties.html
import * as p from "@bokehjs/core/properties"

// The view of the Bokeh extension/ HTML element
// Here you can define how to render the model as well as react to model changes or View events.
export class ChartJSView extends HTMLBoxView {
    model: ChartJS
    objectElement: any // Element

    connect_signals(): void {
        super.connect_signals()

        this.connect(this.model.properties.object.change, () => {
            this.render();
        })
    }

    render(): void {
        super.render()
        this.el.innerHTML = `<button type="button">${this.model.object}</button>`
        this.objectElement = this.el.firstElementChild

        this.objectElement.addEventListener("click", () => {this.model.clicks+=1;}, false)
    }
}

export namespace ChartJS {
    export type Attrs = p.AttrsOf<Props>
    export type Props = HTMLBox.Props & {
        object: p.Property<string>,
        clicks: p.Property<number>,
    }
}

export interface ChartJS extends ChartJS.Attrs { }

// The Bokeh .ts model corresponding to the Bokeh .py model
export class ChartJS extends HTMLBox {
    properties: ChartJS.Props

    constructor(attrs?: Partial<ChartJS.Attrs>) {
        super(attrs)
    }

    static __module__ = "panel.models.chartjs"

    static init_ChartJS(): void {
        this.prototype.default_view = ChartJSView;

        this.define<ChartJS.Props>(({Int, String}) => ({
            object: [String, "Click Me!"],
            clicks: [Int, 0],
        }))
    }
}
```

- Add the ChartJS typescript model to `panel/models/index.ts`

```typescript
export {ChartJS} from "./chartjs"
```

- You can then build the model using `panel build panel`. It should look similar to

```bash
(base) root@475bb36209a9:/workspaces/panel# panel build panel
Working directory: /workspaces/panel/panel
Using /workspaces/panel/panel/tsconfig.json
Compiling styles
Compiling TypeScript (45 files)
Linking modules
Output written to /workspaces/panel/panel/dist
All done.
```

- You can then add a test file `panel/tests/pane/test_chartjs.py` with a test and small app.

```python
import panel as pn


def test_constructor():
    chartjs = pn.pane.ChartJS(object="Click Me Now!")

def get_app():
    chartjs = pn.pane.ChartJS(object="Click Me Now!")
    return pn.Column(
        chartjs, pn.Param(chartjs, parameters=["object", "clicks"])
    )

if __name__.startswith("bokeh"):
    get_app().servable()
```

- Run `pytest panel/tests/pane/test_chartjs.py` and make sure it passes.
- Serve the app with `panel serve panel/tests/pane/test_chartjs.py --auto --show`

You have to *hard refresh* your browser to reload the new panel `.js` files with your `ChartJS` model. In Chrome I press `CTRL+F5`. See [How to hard refresh in Chrome, Firefox and IE](https://www.namecheap.com/support/knowledgebase/article.aspx/10078/2194/how-to-do-a-hard-refresh-in-chrome-firefox-and-ie/) for other browsers.

Now you can manually test your model

![Chart JS Button](../assets/chartjs-button.gif)

- Finally you should save your changes via `git add .` and maybe even commit them `git commit -m "First iteration on ChartJS model"`

## Tips and Tricks for new Custom Extensions Developers

- Work in small increments and stage your changes when they work
- Remember to `panel build panel` and hard refresh before you test.
- Add [console.log](https://www.w3schools.com/jsref/met_console_log.asp) to your `.ts` code initially for debugging.
- Use the [*Developer Console*](https://developers.google.com/web/tools/chrome-devtools) to see the `console.log` output and identify errors. In my browsers I toggle the Developer Tools using `CTRL+SHIFT+I`.
- Find inspiration for next steps in the existing Panel Custom Extensions. For `ChartJS` one of the most relevant extensions would be `Echarts`. See Panel [echarts.py](https://github.com/holoviz/panel/blob/master/panel/pane/echarts.py), Bokeh [echarts.py](https://github.com/holoviz/panel/blob/master/panel/models/echarts.py) and [echarts.ts](https://github.com/holoviz/panel/blob/master/panel/models/echarts.ts).